In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font',family= 'Malgun Gothic')  # 한글설정

import seaborn as sns

In [2]:
import scipy.stats as stats  # 귀무가설

In [3]:
!pip install --user plotly
import plotly.express as px

# 2-2) 기계학습평가(Model Evaluation)
모델 평가 기법(Evaluation)

### 1) 분류에서의 평가

- 정확도 Accuracy = (정확하게 분류한 데이터 수) / (전체 데이터 수)
     - 정확도 평가는 분류 데이터가 한쪽으로 취우친 경우 정상 평가 불가

- 오차행렬 Confusion Matrix: 정확도 분석, 정상 모델인지 확인

- 정밀도 Precision:예측값 중 잘 맞춘 비율 = TP/(FP + TP : Predict Positive)

- 재현율 Recall: 실제 값 중 잘 맞춘 예측 비율 = TP/(FN + TP : Real Positive)


In [4]:
df1 = pd.read_csv('01_Data.csv')

In [5]:
df1.head()

,Index,Member_ID,Sales_Type,Contract_Type,Channel,Datetime,Term,Payment_Type,Product_Type,Amount_Month,Customer_Type,Age,Address1,Address2,State,Overdue_count,Overdue_Type,Gender,Credit_Rank,Bank
0,1,66758234,렌탈,일반계약,영업방판,2019-05-06,60,CMS,DES-1,96900,개인,42.0,경기도,경기도,계약확정,0,없음,여자,9.0,새마을금고
1,2,66755948,렌탈,교체계약,영업방판,2020-02-20,60,카드이체,DES-1,102900,개인,39.0,경기도,경기도,계약확정,0,없음,남자,2.0,현대카드
2,3,66756657,렌탈,일반계약,홈쇼핑/방송,2019-02-28,60,CMS,DES-1,96900,개인,48.0,경기도,경기도,계약확정,0,없음,여자,8.0,우리은행
3,4,66423450,멤버십,멤버십3유형,재계약,2019-05-13,12,CMS,DES-1,66900,개인,39.0,경기도,경기도,계약확정,0,없음,남자,5.0,농협회원조합
4,5,66423204,멤버십,멤버십3유형,재계약,2019-05-10,12,CMS,DES-1,66900,개인,60.0,경기도,경기도,기간만료,12,있음,남자,8.0,농협회원조합


In [6]:
df1['Overdue_Type'].value_counts()

없음    49110
있음     2191
Name: Overdue_Type, dtype: int64

---------------
# 1. 데이터 전처리

In [7]:
# 1. 데이터 전처리

# 결측치 처리_ [제거] dropna(): 행 내 결측값이 하나라도 존재하면 해당 행을 삭제

df1_clean = df1.dropna()

-----------
# 2. X와 Y를 선택

In [8]:
# 2. X와 Y를 선택

X = df1_clean[['Term', 'Amount_Month', 'Credit_Rank', 'Age']]
Y = df1_clean['Overdue_Type']

-----------------
# 3. 학습데이터와 검증데이터 분할
                                                     (train/test set 구분)

In [9]:
# 3. 학습데이터와 검증데이터 분할(train/test set 구분)

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.3,
                                                   random_state = 1234)


-------------
# 4. 학습 실시

In [10]:

from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()  # tree모델 생성
model.fit(X_train, Y_train)  # 학습 실시

DecisionTreeClassifier()

----------
# 5. 평가

In [11]:
from sklearn.metrics import classification_report

Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)

### 5.1) 학습 성능

In [12]:
print(classification_report(Y_train, Y_train_pred))

              precision    recall  f1-score   support

          없음       0.97      1.00      0.99     27423
          있음       0.94      0.27      0.42      1029

    accuracy                           0.97     28452
   macro avg       0.96      0.63      0.70     28452
weighted avg       0.97      0.97      0.97     28452



### 5.2) 일반화 성능

In [13]:
print(classification_report(Y_test, Y_test_pred))

              precision    recall  f1-score   support

          없음       0.97      0.99      0.98     11795
          있음       0.24      0.10      0.14       400

    accuracy                           0.96     12195
   macro avg       0.60      0.54      0.56     12195
weighted avg       0.95      0.96      0.95     12195



# 2-3) 특성공학_Feature Engineering
수식화 (회귀분석 -> 기계학습(데이터 마이닝))

- **특성공학 (Feature Engineering)**


- **1. scaling & Encoding**


    - scaling: 연속형 숫자데이터의 Scale 맞춰주는 작업
        - Standard Scaler: 평균 0/ 표준편차 1
        - Miin Max Scaler: 최솟값 0/ 최댓값 1
        - Roberst Scaler: 중앙값 0/ IQR 1
    
    
    - Encoding: 범주형 데이터를 숫자형태로 변환
        - Label Encoding: 범주형 데이터의 각 항목값을 정수로 변환
        - One Hot Encoding: 각 범주를 column으로 변환하여 해당 값만 1을 줌

# Pipe Line 파이프라인 생성

In [14]:
!pip install imblearn

In [15]:
from imblearn.pipeline import make_pipeline 
                                    # 특성공학 +학습 파이프 생성 함수
    
from sklearn.preprocessing import MinMaxScaler
                                    # 숫자 스케일링(0 ~ 1)

from sklearn.preprocessing import OneHotEncoder
                                    # 문자 인코딩 (항목 -> 1/0)
    
from sklearn.compose import make_column_transformer
                                    # 각 항목의 타입별로 파이프 분할

from sklearn.impute import SimpleImputer    
                                    # 결측치 단순대치

### 숫자 데이터 (결측값을 평균으로 처리 -> 스케일링)

In [16]:
numeric_pipe = make_pipeline((SimpleImputer(strategy='mean')),
                            (MinMaxScaler()))

numeric_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('minmaxscaler', MinMaxScaler())])

### 문자 데이터 (결측값을 최빈값 처리 -> 인코딩)

In [17]:
category_pipe = make_pipeline((SimpleImputer(strategy='most_frequent')),
                             (OneHotEncoder()))
category_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder', OneHotEncoder())])

## 파이프라인 구축
### 숫자 항목은 숫자끼리 문자항목은 문자끼리 분류하여 처리

In [18]:
x = df1[['Term', 'Amount_Month', 'Age', 'Credit_Rank', 'Gender', 'Product_Type']]
y = df1['Overdue_Type']

In [19]:
numeric_list = ['Term', 'Amount_Month', 'Age', 'Credit_Rank']
category_list = ['Gender', 'Product_Type']
preprocesing_pipe_w1 = make_column_transformer((numeric_pipe, numeric_list),
                                           (category_pipe, category_list))

preprocesing_pipe_w1

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('minmaxscaler',
                                                  MinMaxScaler())]),
                                 ['Term', 'Amount_Month', 'Age',
                                  'Credit_Rank']),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder())]),
                                 ['Gender', 'Product_Type'])])

In [20]:
X = df1[['Term', 'Amount_Month', 'Age', 'Credit_Rank', 'Gender', 'Product_Type']]
Y = df1['Overdue_Type']

In [21]:
number_list = X.describe().columns.tolist()
category_list = X.describe(include='object').columns.tolist()
preprocessing_pipe_w2 = make_column_transformer( (numeric_pipe, number_list),
                                              (category_pipe, category_list))
preprocessing_pipe_w2

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('minmaxscaler',
                                                  MinMaxScaler())]),
                                 ['Term', 'Amount_Month', 'Age',
                                  'Credit_Rank']),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder())]),
                                 ['Gender', 'Product_Type'])])

### 전처리 파이프 + 학습 알고리즘

In [23]:
model_pipe = make_pipeline(preprocessing_pipe_w2, DecisionTreeClassifier())
model_pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('minmaxscaler',
                                                                   MinMaxScaler())]),
                                                  ['Term', 'Amount_Month',
                                                   'Age', 'Credit_Rank']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   On

In [24]:
model_pipe.fit(X, Y)  # 학습수행

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('minmaxscaler',
                                                                   MinMaxScaler())]),
                                                  ['Term', 'Amount_Month',
                                                   'Age', 'Credit_Rank']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   On

In [26]:
import pickle

In [27]:
pickle.dump(model_pipe, open('model.sav','wb')) # model.sav 저장